<a href="https://colab.research.google.com/github/mukulre/Projects/blob/main/Pneumonia_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils import class_weight
from sklearn.metrics import (
    precision_recall_curve,
    roc_curve,
    roc_auc_score,
    confusion_matrix,
    ConfusionMatrixDisplay
)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Pneumonia Detection/chest_xray/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)
val_generator = val_test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Pneumonia Detection/chest_xray/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)
test_generator = val_test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Pneumonia Detection/chest_xray/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# Class weights
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

# Load ResNet50 base model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

# Define the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    class_weight=class_weights,
    callbacks=[early_stopping],
    verbose=1
)

# Fine-tuning
base_model.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
history_fine = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    class_weight=class_weights,
    callbacks=[early_stopping],
    verbose=1
)

# Save model
model.save('pneumonia_model_resnet50.h5')

# Combine training history
def combine_histories(hist1, hist2):
    combined = {}
    for key in hist1.history:
        combined[key] = hist1.history[key] + hist2.history.get(key, [])
    return combined

combined_history = combine_histories(history, history_fine)

# Plot training metrics
def plot_training_metrics(history_dict):
    epochs = range(1, len(history_dict['accuracy']) + 1)
    plt.figure(figsize=(14, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, history_dict['accuracy'], label='Train Accuracy', color='blue')
    plt.plot(epochs, history_dict['val_accuracy'], label='Val Accuracy', color='green')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(epochs, history_dict['loss'], label='Train Loss', color='red')
    plt.plot(epochs, history_dict['val_loss'], label='Val Loss', color='orange')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

plot_training_metrics(combined_history)

# Evaluate predictions
y_true = test_generator.classes
y_pred_prob = model.predict(test_generator).ravel()
y_pred_class = (y_pred_prob > 0.5).astype(int)

# ROC Curve
fpr, tpr, _ = roc_curve(y_true, y_pred_prob)
auc_score = roc_auc_score(y_true, y_pred_prob)
plt.figure()
plt.plot(fpr, tpr, label=f"AUC = {auc_score:.2f}")
plt.plot([0, 1], [0, 1], 'k--')
plt.title("ROC Curve")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.grid(True)
plt.show()

# Precision-Recall Curve
precision, recall, _ = precision_recall_curve(y_true, y_pred_prob)
plt.figure()
plt.plot(recall, precision, marker='.')
plt.title("Precision-Recall Curve")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.grid(True)
plt.show()

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred_class)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Normal', 'Pneumonia'])
disp.plot(cmap='Blues')
plt.title("Confusion Matrix")
plt.show()

# Model Comparison Graph
models = ['DenseNet121', 'InceptionV3', 'ResNet50', 'VGG16', 'EfficientNetB0', 'CBAM']
accuracies = [0.95, 0.96, 0.94, 0.92, 0.96, 0.97]  # Replace with actual
auc_scores = [0.98, 0.98, 0.97, 0.96, 0.98, 0.99]   # Replace with actual

x = np.arange(len(models))
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.bar(x, accuracies, color='skyblue')
plt.xticks(x, models, rotation=45)
plt.ylabel('Accuracy')
plt.title('Model Accuracy Comparison')

plt.subplot(1, 2, 2)
plt.bar(x, auc_scores, color='salmon')
plt.xticks(x, models, rotation=45)
plt.ylabel('AUC Score')
plt.title('Model AUC Comparison')

plt.tight_layout()
plt.show()

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


KeyboardInterrupt: 